# BERT의 이해와 간단한 활용



In [1]:
# 감성 분석
from transformers import pipeline # conda로 가상환경에 transformers를 설치

clf = pipeline("sentiment-analysis")
result = clf("what a beautiful day!")[0]
print("감성분석 결과: %s, 감성스코어: %0.4f" % (result['label'], result['score']))

/Users/jungnerd/miniforge3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 629/629 [00:00<00:00, 105kB/s]
Downloading: 100%|██████████| 268M/268M [00:25<00:00, 10.6MB/s] 
All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you

감성분석 결과: POSITIVE, 감성스코어: 0.9999


In [2]:
# GPT-2를 이용해 문서 생성을 재미삼아 해보자

from transformers import pipeline

text_generator = pipeline("text-generation")
result = text_generator("Alice was beginning to get very tired of sitting by her sister on the bank,")
print(result[0]['generated_text'])

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.
Downloading: 100%|██████████| 665/665 [00:00<00:00, 252kB/s]
Downloading: 100%|██████████| 498M/498M [00:46<00:00, 10.7MB/s]   
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.
Downloading: 100%|██████████| 1.04M/1.04M [00:01<00:00, 778kB/s] 
Downloading: 100%|██████████| 456k/456k [00:00<00:00, 473kB/s]  
Downloading: 100%|██████████| 1.36M/1.36M [00:01<00:00, 1.13MB/s]
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence
/Users/jungnerd/miniforge3/envs/nlp/lib/python3.9/site-packages/transformers/generatio

Alice was beginning to get very tired of sitting by her sister on the bank, hoping that she would find something to eat, and her brother would get her some. The brothers got to work and were late for dinner, so Mom stopped by and gave


## 14.5 자동 클래스를 이용한 토크나이저와 모형의 사용



In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased-finetuned-mrpc")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased-finetuned-mrpc")

# 의미적으로 유사한 두 문장을 선언
input_sentence = "She angered me with her inappropriate comments, rumor-spreading, and disrespectfulness at the formal dinner table"
target_sequence = "She made me angry when she was rude at dinner"

# 토큰화
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
# 모형으로 결과를 예측
logits = model(**tokens).logits
# 소프트맥스를 이용해 결과값을 클래스에 대한 확률로 변환
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

/Users/jungnerd/miniforge3/envs/nlp/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|██████████| 433M/433M [00:46<00:00, 9.32MB/s] 


no: 29%
yes: 71%


In [4]:
target_sequence = "The boy quickly ran across the finish line, seizing yet another victory"
tokens = tokenizer(input_sentence, target_sequence, return_tensors="pt")
logits = model(**tokens).logits
results = torch.softmax(logits, dim=1).tolist()[0]

for i, label in enumerate(['no', 'yes']):
    print(f"{label}: {int(round(results[i] * 100))}%")

no: 95%
yes: 5%


In [5]:
import nltk
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split #sklearn에서 제공하는 split 함수를 사용
import numpy as np

nltk.download('movie_reviews')
fileids = movie_reviews.fileids() #movie review data에서 file id를 가져옴
reviews = [movie_reviews.raw(fileid) for fileid in fileids] #file id를 이용해 raw text file을 가져옴
categories = [movie_reviews.categories(fileid)[0] for fileid in fileids] 

# label을 0, 1의 값으로 변환
label_dict = {'pos':1, 'neg':0}
y = np.array([label_dict[c] for c in categories])

X_train, X_test, y_train, y_test = train_test_split(reviews, y, test_size=0.2, random_state=7)
print('Train set count: ', len(X_train))
print('Test set count: ', len(X_test))

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\ParkSangun\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Train set count:  1600
Test set count:  400


In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

# cuda를 이용한 GPU 연산이 가능하면 cuda를 사용하고, 아니면 cpu를 사용
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# Auto Classes를 이용해 사전학습된 내용에 맞는 토크나이저와 모형을 자동으로 설정
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# 모델을 gpu로 옮겨서 연산을 준비
model = model.to(device)

batch_size = 10 # 모형으로 한번에 예측할 데이터의 수
y_pred = [] # 전체 예측결과를 저장

num_batch = len(y_test)//batch_size

for i in range(num_batch):
    inputs = tokenizer(X_test[i*batch_size:(i+1)*batch_size], truncation=True, padding=True, return_tensors="pt")
    # 토큰화 결과를 GPU로 이동
    inputs = inputs.to(device)
    # 모형으로 결과를 예측
    logits = model(**inputs).logits
    # 결과값을 클래스에 대한 확률로 변환
    pred = F.softmax(logits, dim=-1)
    # 예측결과를 CPU로 가져와서 넘파이로 변환한 후, argmax로 확률이 가장 큰 클래스를 선택함
    results = pred.cpu().detach().numpy().argmax(axis=1)
    
    # 전체 예측결과에 추가
    y_pred.extend(results.tolist())

# gpu 메모리를 비움
torch.cuda.empty_cache()

score = sum(y_test == np.array(y_pred))/len(y_test)
print("NLTK 영화리뷰 감성분석 정확도:", score)

NLTK 영화리뷰 감성분석 정확도: 0.8425
